In [1]:
import os
os.chdir('F:\machine learning\lab 2\lab 7')

<>:2: SyntaxWarning: invalid escape sequence '\m'
<>:2: SyntaxWarning: invalid escape sequence '\m'
C:\Users\Admin\AppData\Local\Temp\ipykernel_14280\478413239.py:2: SyntaxWarning: invalid escape sequence '\m'
  os.chdir('F:\machine learning\lab 2\lab 7')


In [2]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
#from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
from keras.callbacks import Callback

In [19]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

In [24]:
def create_lstm():
    input_data = Input(shape=(time_steps, num_features))
    lstm_layer1 = LSTM(8, return_sequences=True)(input_data)
    lstm_layer2 = LSTM(20)(lstm_layer1)
    x = Flatten()(lstm_layer2)
    output_data = Dense(1)(x)
    model = Model(input_data, output_data)
    return model

In [26]:
model1 = create_lstm()
model1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 24, 21)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 24, 8)               │             960 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 20)                  │           2,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 20)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              21 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,301 (12.89 KB)

 Trainable params: 3,301 (12.89 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
tensorflow.keras.utils.plot_model(model1 )

NameError: name 'model1' is not defined

In [ ]:
checkpoints = r'C:\F:\machine learning\lab 2\lab 7\lab_10\check_points'
OUTPUT_PATH = r'F:\machine learning\lab 2\lab 7\lab_10'
FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])

In [ ]:
from keras.callbacks import ModelCheckpoint

# Corrected filepath with .keras extension
checkpoints = r"C:\F\machine learning\lab 2\lab 7\lab_10\check_points\best_model.keras"

# construct the callback to save only the *best* model to disk
EpochCheckpoint1 = ModelCheckpoint(filepath=checkpoints,
                                   monitor="val_loss",
                                   save_best_only=True, 
                                   verbose=1)

# construct the set of callbacks
callbacks = [EpochCheckpoint1]


In [47]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model =create_lstm()
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [49]:
import os
path_dataset =(r'F::\machine learning\lab 2\lab 7\1_Orignal_AEP_hourly.csv')
path_tr = os.path.join(path_dataset, 'F:\machine learning\lab 2\lab 7\AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'F:\machine learning\lab 2\lab 7\AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'F:\machine learning\lab 2\lab 7\AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'F:\machine learning\lab 2\lab 7\AEP_scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

<>:3: SyntaxWarning: invalid escape sequence '\m'
<>:6: SyntaxWarning: invalid escape sequence '\m'
<>:9: SyntaxWarning: invalid escape sequence '\m'
<>:13: SyntaxWarning: invalid escape sequence '\m'
<>:3: SyntaxWarning: invalid escape sequence '\m'
<>:6: SyntaxWarning: invalid escape sequence '\m'
<>:9: SyntaxWarning: invalid escape sequence '\m'
<>:13: SyntaxWarning: invalid escape sequence '\m'
C:\Users\Admin\AppData\Local\Temp\ipykernel_14304\1557545470.py:3: SyntaxWarning: invalid escape sequence '\m'
  path_tr = os.path.join(path_dataset, 'F:\machine learning\lab 2\lab 7\AEP_train.csv')
C:\Users\Admin\AppData\Local\Temp\ipykernel_14304\1557545470.py:6: SyntaxWarning: invalid escape sequence '\m'
  path_v = os.path.join(path_dataset, 'F:\machine learning\lab 2\lab 7\AEP_validation.csv')
C:\Users\Admin\AppData\Local\Temp\ipykernel_14304\1557545470.py:9: SyntaxWarning: invalid escape sequence '\m'
  path_te = os.path.join(path_dataset, 'F:\machine learning\lab 2\lab 7\AEP_test.csv'

((84907, 21), (24259, 21), (12130, 21))

In [51]:
time_steps=24
num_features=21

In [53]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 1.3992893695831299 sec


In [21]:
import numpy as np
import pandas as pd
import time
import pickle
import os

# Fix dataset path
path_dataset = r'F:\machine learning\lab 2\lab 7'

# File paths
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
path_te = os.path.join(path_dataset, 'AEP_test.csv')
path_scaler = os.path.join(path_dataset, 'AEP_scaler.pkl')

# Load data
df_tr = pd.read_csv(path_tr)
df_v = pd.read_csv(path_v)
df_te = pd.read_csv(path_te)
scaler = pickle.load(open(path_scaler, 'rb'))

train_set = df_tr.values
validation_set = df_v.values
test_set = df_te.values

# Define the function if not already
def univariate_multi_step(data, time_steps, target_col=0, target_len=1):
    X, y = [], []
    for i in range(len(data) - time_steps - target_len + 1):
        X.append(data[i:(i + time_steps)])
        y.append(data[(i + time_steps):(i + time_steps + target_len), target_col])
    return np.array(X), np.array(y)

# Prepare data
time_steps = 24
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0, target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0, target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0, target_len=1)
print('Time Consumed', time.time()-start, "sec")

# Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential([
    Input(shape=(train_X.shape[1], train_X.shape[2])),
    LSTM(64),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

callbacks = [EarlyStopping(patience=5)]

# Fit
epochs = 20
verbose = 1
batch_size = 32

history = model.fit(train_X,
                    train_y,
                    batch_size=batch_size,   
                    epochs=epochs, 
                    validation_data=(validation_X, validation_y),
                    callbacks=callbacks,
                    verbose=verbose)


C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Time Consumed 0.4202003479003906 sec
Epoch 1/20
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 34s 12ms/step - loss: 0.0039 - val_loss: 4.7848e-04
Epoch 2/20
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 31s 12ms/step - loss: 3.7315e-04 - val_loss: 1.4946e-04
Epoch 3/20
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 31s 12ms/step - loss: 2.0204e-04 - val_loss: 1.4481e-04
Epoch 4/20
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 31s 12ms/step - loss: 1.6327e-04 - val_loss: 1.3194e-04
Epoch 5/20
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 35s 13ms/step - loss: 1.2934e-04 - val_loss: 1.1664e-04
Epoch 6/20
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 37s 12ms/step - loss: 1.2391e-04 - val_loss: 1.0159e-04
Epoch 7/20
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 31s 11ms/step - loss: 1.0568e-04 - val_loss: 9.0677e-05
Epoch 8/20
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 32s 12ms/step - loss: 1.0049e-04 - val_loss: 1.1281e-04
Epoch 9/20
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 34s 13ms/step - loss: 9.2090e-05 - val_loss: 1.0662e-04
Epoch 10/20
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 39s 12ms/step - loss: 9.1388e-05 - val_los

In [23]:

model = load_model(r'F:\machine learning\lab 2\lab 7\check_points\best_model.keras')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
Mean Absolute Error (MAE): 103.05
Median Absolute Error (MedAE): 80.93
Mean Squared Error (MSE): 18861.9
Root Mean Squared Error (RMSE): 137.34
Mean Absolute Percentage Error (MAPE): 0.71 %
Median Absolute Percentage Error (MDAPE): 0.56 %


y_test_unscaled.shape=  (12106, 1)
y_pred.shape=  (12106, 1)


In [25]:
checkpoints = r'F:\machine learning\lab 2\lab 7\check_points\best_model.keras'
model_path = r'F:\machine learning\lab 2\lab 7\check_points\best_model.keras'
model = None
start_epoch= 34

In [45]:
# construct the callback to save only the *best* model to disk
EpochCheckpoint1 = ModelCheckpoint(
    filepath=checkpoints,
    monitor="val_loss",
    save_best_only=True, 
    verbose=1
)

# construct the set of callbacks
callbacks = [EpochCheckpoint1]

# check if the model file exists
import os
if not os.path.exists(model_path):
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(1e-3)
    model.compile(loss='mae', optimizer=opt, metrics=["mae", "mape"])
else:
    print(f"[INFO] loading model from: {model_path}")
    model = load_model(model_path)

    # update the learning rate
    print(f"[INFO] old learning rate: {K.get_value(model.optimizer.learning_rate)}")
    K.set_value(model.optimizer.learning_rate, 1e-4)
    print(f"[INFO] new learning rate: {K.get_value(model.optimizer.learning_rate)}")


[INFO] loading model from: F:\machine learning\lab 2\lab 7\check_points\best_model.keras
[INFO] old learning rate: 9.999999747378752e-05


AttributeError: 'str' object has no attribute 'name'

In [20]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

Epoch 1/10
2653/2653 [==============================] - ETA: 0s - loss: 0.0058 - mae: 0.0058 - mape: 14.8268
Epoch 1: val_loss improved from inf to 0.00590, saving model to C:\Users\Administrator\Downloads\ML Lab\checkpoint\ML Lab\lab10\E2-cp-0001-loss0.01.h5
2653/2653 [==============================] - 58s 21ms/step - loss: 0.0058 - mae: 0.0058 - mape: 14.8268 - val_loss: 0.0059 - val_mae: 0.0059 - val_mape: 2.4532
Epoch 2/10
2653/2653 [==============================] - ETA: 0s - loss: 0.0058 - mae: 0.0058 - mape: 30.0224
Epoch 2: val_loss did not improve from 0.00590
2653/2653 [==============================] - 60s 23ms/step - loss: 0.0058 - mae: 0.0058 - mape: 30.0224 - val_loss: 0.0060 - val_mae: 0.0060 - val_mape: 2.4553
Epoch 3/10
2652/2653 [============================>.] - ETA: 0s - loss: 0.0058 - mae: 0.0058 - mape: 14.1057
Epoch 3: val_loss did not improve from 0.00590
2653/2653 [==============================] - 58s 22ms/step - loss: 0.0058 - mae: 0.0058 - mape: 14.1030 - va

In [33]:

model = load_model(r'F:\machine learning\lab 2\lab 7\check_points\best_model.keras')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step
Mean Absolute Error (MAE): 103.05
Median Absolute Error (MedAE): 80.93
Mean Squared Error (MSE): 18861.9
Root Mean Squared Error (RMSE): 137.34
Mean Absolute Percentage Error (MAPE): 0.71 %
Median Absolute Percentage Error (MDAPE): 0.56 %


y_test_unscaled.shape=  (12106, 1)
y_pred.shape=  (12106, 1)
